# Introduction
Halo teman-teman, disini kami dari kelompok ... akan mengerjakan project data analyst tentang Sistem Rekomendasi Trip di Indonesia. Data yang digunakan bersumber dari Kaggle yaitu 'Indonesia Tourism Destination' yang terdiri dari 4 files .csv yang disimpan didalam folder assets pada GDrive yang kami gunakan. Berikut ini penjelasan dari setiap file yang kami gunakan:


1.   tourism_with_id.csv: Informasi tentang tempat wisata (misalnya nama, lokasi, deskripsi).

2.   user.csv: Data pengguna dummy untuk membuat fitur rekomendasi berdasarkan pengguna (misalnya ID pengguna, preferensi).

3.   tourism_rating.csv: Rating tempat wisata yang diberikan oleh pengguna.

4.   package_tourism.csv: Rekomendasi tempat wisata terdekat berdasarkan waktu, biaya, dan rating.

Model yang akan digunakan adalah model Hybrid dimana menggabungkan 2 model, yaitu Content-based Filtering Model dan Collaborative Filtering Model dengan cara kerja seperti berikut:

*   Content-based Filtering digunakan untuk merekomendasikan tempat wisata berdasarkan fitur seperti maskapai, jumlah berhenti, dan durasi.
*   Collaborative Filtering untuk memperhitungkan preferensi pengguna serupa dalam memberikan rekomendasi.

# System Preparation
pada bagian ini kita menyiapkan modul modul yang akan kita gunakan pada program kali ini, dan diatas adalah beberapa penginstalan modul yang kita lakukan dengan menggunakan kondisi dimana jika modul sudah terinstall maka tidak perlu kita lakukan install, dan jika belum maka akan di install secara automatis.

In [38]:
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [ ]:
# Mengecek dan menginstal modul surprise
if 'google.colab' in str(get_ipython()):
    try:
        import surprise
        print("Module 'surprise' sudah terinstall")
    except ImportError:
        !pip install surprise
else:
    print("Module 'surprise' sudah terinstall")

# Mengecek dan menginstal modul pandas
if 'google.colab' in str(get_ipython()):
    try:
        import pandas as pd
        print("Module 'pandas' sudah terinstall")
    except ImportError:
        !pip install pandas
else:
    print("Module 'pandas' sudah terinstall")

# Mengecek dan menginstal modul scikit-learn
if 'google.colab' in str(get_ipython()):
    try:
        import sklearn
        print("Module 'scikit-learn' sudah terinstall")
    except ImportError:
        !pip install scikit-learn
else:
    print("Module 'scikit-learn' sudah terinstall")

# Mengecek dan menginstal modul numpy
if 'google.colab' in str(get_ipython()):
    try:
        import numpy as np
        print("Module 'numpy' sudah terinstall")
    except ImportError:
        !pip install numpy
else:
    print("Module 'numpy' sudah terinstall")

# Mengecek dan menginstal modul matplotlib
if 'google.colab' in str(get_ipython()):
    try:
        import matplotlib.pyplot as plt
        print("Module 'matplotlib' sudah terinstall")
    except ImportError:
        !pip install matplotlib
else:
    print("Module 'matplotlib' sudah terinstall")

# Mengecek dan menginstal modul seaborn
if 'google.colab' in str(get_ipython()):
    try:
        import seaborn as sns
        print("Module 'seaborn' sudah terinstall")
    except ImportError:
        !pip install seaborn
else:
    print("Module 'seaborn' sudah terinstall")


Module 'surprise' sudah terinstall
Module 'pandas' sudah terinstall
Module 'scikit-learn' sudah terinstall
Module 'numpy' sudah terinstall
Module 'matplotlib' sudah terinstall
Module 'seaborn' sudah terinstall


# Data Requirement
pada bagian ini kita mengumpulkan data data yang akan kita butuhkan, dan kita akan baca menggunakan pandas pada program kali ini, dan langkah pertama yang kita lakukan adalah mount drive untuk mengakses file assets kita, setelah itu kita lakukan pembacaan file csv menggunakan pandas dan kita lakukan pengechekan untuk setiap isi file menggunakan salah satu fungsi pandas yaitu head, yang dimana kita akan menampilkan 5 data pertama (defaults) pada setiap file yang sudah kita baca menggunakan pa

# Import Module
pada bagian ini modul modul yang kita akan gunakan sudah terinstall maka kita akan melakukan import modul sudah kita install dan akan kita gunakan.

In [39]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Menyimpan masing-masing dataset kedalam variable

In [42]:
file_path_package = '/content/drive/MyDrive/Indonesia-Trip-Recomendation/Assets/package_tourism.csv'
file_path_TRating = '/content/drive/MyDrive/Indonesia-Trip-Recomendation/Assets/tourism_rating.csv'
file_path_TWId = '/content/drive/MyDrive/Indonesia-Trip-Recomendation/Assets/tourism_with_id.csv'
file_path_user = '/content/drive/MyDrive/Indonesia-Trip-Recomendation/Assets/user.csv'

In [44]:
df_package = pd.read_csv(file_path_package)
df_package.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Indonesia-Trip-Recomendation/Assets/package_tourism.csv'

Data Rating

In [ ]:
df_TRating = pd.read_csv(file_path_TRating)
df_TRating.head()

,User_Id,Place_Id,Place_Ratings
0,1,179,3
1,1,344,2
2,1,5,5
3,1,373,3
4,1,101,4


In [ ]:
df_user = pd.read_csv(file_path_user)
df_user.head()

,User_Id,Location,Age
0,1,"Semarang, Jawa Tengah",20
1,2,"Bekasi, Jawa Barat",21
2,3,"Cirebon, Jawa Barat",23
3,4,"Bekasi, Jawa Barat",21
4,5,"Lampung, Sumatera Selatan",20


In [ ]:
df_TWId = pd.read_csv(file_path_TWId)
df_TWId.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Unnamed: 11,Unnamed: 12
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,NaN,1
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,NaN,2
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,NaN,3
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,106.895156,NaN,4
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134,NaN,5
